# annual analytics 2025-2026

this notebook reads the csv packs generated by analytics-range and report-range.

expected folders:
- data/reports/analytics_2025-01-01_2026-01-20/
- data/reports/range_2025-01-01_2026-01-20/


In [ ]:
from pathlib import Path
import pandas as pd

start = "2025-01-01"
end = "2026-01-20"

reports_dir = Path("data/reports")
analytics_dir = reports_dir / f"analytics_{start}_{end}"
range_dir = reports_dir / f"range_{start}_{end}"

analytics_dir, range_dir


In [ ]:
quality = pd.read_csv(analytics_dir / "quality_daily.csv")
seasonality = pd.read_csv(analytics_dir / "seasonality_uf.csv")
hotspots = pd.read_csv(analytics_dir / "hotspots_mun_period.csv")
shifts = pd.read_csv(analytics_dir / "top_shifts_uf.csv")

br_daily = pd.read_csv(range_dir / "br_daily.csv")
uf_top = pd.read_csv(range_dir / "uf_top.csv")
mun_top = pd.read_csv(range_dir / "mun_top.csv")

quality.head()


In [ ]:
uf_top.head(10)

mun_top.head(10)


In [ ]:
import matplotlib.pyplot as plt

quality["day"] = pd.to_datetime(quality["day"])

fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)
axes[0].plot(quality["day"], quality["pct_com_mun"])
axes[0].set_ylabel("pct_com_mun")
axes[1].bar(quality["day"], quality["missing_mun"])
axes[1].set_ylabel("missing_mun")
axes[1].set_xlabel("day")
plt.tight_layout()


In [ ]:
br_daily["day"] = pd.to_datetime(br_daily["day"])

fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(br_daily["day"], br_daily["n_focos_total"], label="total")
ax.plot(br_daily["day"], br_daily["n_focos_com_mun"], label="com_mun")
ax.set_ylabel("n_focos")
ax.legend()
plt.tight_layout()


In [ ]:
seasonality["month"] = pd.to_datetime(seasonality["month"])
top_ufs = (
    seasonality.groupby("uf")["n_focos"]
    .sum()
    .sort_values(ascending=False)
    .head(10)
    .index
    .tolist()
)

fig, ax = plt.subplots(figsize=(12, 5))
for uf in top_ufs:
    data = seasonality[seasonality["uf"] == uf].sort_values("month")
    ax.plot(data["month"], data["n_focos"], label=uf)

ax.legend(ncol=2, fontsize=8)
ax.set_ylabel("n_focos")
plt.tight_layout()


In [ ]:
hotspots.head(20)


In [ ]:
shifts
